# Key Terminology [](url)
## Munging

In [0]:
from pyspark.sql.session import SparkSession
spark = SparkSession.builder.appName("Spark DataFrames").getOrCreate()

#Important passive Munging - EDA of schema/structure functions we can use

In [0]:
dbutils.fs.ls("/Volumes/lakehouse1/dbread/read_volume/commondata/")

In [0]:
display(
    spark.read.text("/Volumes/lakehouse1/dbread/read_volume/commondata/custsmodified")
)


In [0]:
df1 = spark.read.csv("/Volumes/lakehouse1/dbread/read_volume/commondata/custsmodified", header=False, inferSchema=True).toDF("id", "name", "lname", "age", "prof")
display(df1)
# df1.printSchema()
# display([col for col in df1.columns])
# print(df1.schema)
# display(df1.dtypes)
# display(df1.describe())
# display(df1.summary())


In [0]:
print("actual count of the data",df1.count())
print("de-duplicated record (all columns) count",df1.distinct().count())#de duplicate the entire columns of the given  dataframe
print("de-duplicated record (all columns) count",df1.dropDuplicates().count())#de duplicate the entire columns of the given  dataframe
print("de-duplicated given cid column count",df1.dropDuplicates(['id']).count())#de duplicate the entire columns of the given  dataframe
display(df1.describe())
display(df1.summary())

In [0]:
#1. Single file
struct1="id string, firstname string, lastname string, age string, profession string"
rawdf1=spark.read.schema(struct1).csv(path="/Volumes/lakehouse1/dbread/read_volume/commondata/dummy1/custs")
#display(rawdf1.count())

rawdf1=spark.read.schema(struct1).csv(path=["/Volumes/lakehouse1/dbread/read_volume/commondata/custsmodified","/Volumes/lakehouse1/dbread/read_volume/commondata/custsmodified_NY","/Volumes/lakehouse1/dbread/read_volume/commondata/dummy1/custs"],pathGlobFilter="custs*",recursiveFileLookup=True)
display(rawdf1.count())

In [0]:
strt1="id string, firstname string, lastname string, age string, profession string"
rawdf1=spark.read.schema(strt1).csv(path=["/Volumes/lakehouse1/dbread/read_volume/commondata/"],recursiveFileLookup=True,pathGlobFilter="custsmodified_N*")
display(rawdf1.count())
display(rawdf1)

strt2="id string, firstname string, age string, profession string,city string"
rawdf2=spark.read.schema(strt1).csv(path=["/Volumes/lakehouse1/dbread/read_volume/commondata/"],recursiveFileLookup=True,pathGlobFilter="custsmodified_T*")
display(rawdf2.count())
display(rawdf2)

In [0]:
rawdf_merged=rawdf1.unionByName(rawdf2,allowMissingColumns=True)
display(rawdf_merged)

###Combining Data + Schema Evolution/Merging (Structuring) - Preliminary Datamunging


####**Single File**

In [0]:
struct1="id string, firstname string, lastname string, age string, profession string"
rawdf1=spark.read.schema(struct1).csv(path="/Volumes/lakehouse1/dbread/read_volume/custsmodified")
print(f"Single file total count",rawdf1.count())
#display(rawdf1)

#Multiple files (with different names)
rawdf1=spark.read.schema(struct1).csv(path=["/Volumes/lakehouse1/dbread/read_volume/custsmodified","/Volumes/lakehouse1/dbread/read_volume/custsmodified_NY","/Volumes/lakehouse1/dbread/read_volume/sub/custsmodified_TX"])
print(f"Multiple files (with different names)",rawdf1.count())
#display(rawdf1)

#Multiple files (with different names, recursive)
rawdf1 = spark.read.schema(struct1).csv(
    path="/Volumes/lakehouse1/dbread/read_volume/",
    pathGlobFilter="custs*",
    recursiveFileLookup=True
)
print(f"Multiple files (with different names, recursive)",rawdf1.count())
display(rawdf1)



In [0]:
schem1 = "id string, firstname string, lastname string, age int, profession string"
rawdf1=spark.read.csv("/Volumes/lakehouse1/dbread/read_volume/sub/custsmodified_NY",schema=schem1)
display(rawdf1)

schem2 = "id string, firstname string, age int, profession string, city string"
rawdf2 =spark.read.csv("/Volumes/lakehouse1/dbread/read_volume/sub/custsmodified_TX",schema=schem2)
display(rawdf2)

rawdf_merged= rawdf1.unionByName(rawdf2, allowMissingColumns=True)
display(rawdf_merged)
#rawdf_merged=rawdf1.unionByName(rawdf2